# 多输入多输出通道

实现一下多输入通道互相关运算

In [1]:
import sys
sys.path.append('..')

In [2]:
from d2l import mindspore as d2l

def corr2d_multi_in(X, K):
    return sum(d2l.corr2d(x, k) for x, k in zip(X, K))

验证互相关运算的输出

In [3]:
X = d2l.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
               [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = d2l.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])

corr2d_multi_in(X, K)

[WARNING] KERNEL(4100470,7f5f2f443740,python):2021-11-08-12:57:24.272.112 [mindspore/ccsrc/backend/kernel_compiler/gpu/gpu_kernel_factory.cc:96] ReducePrecision] Kernel [TensorScatterUpdate] does not support int64, cast input 1 to int32.
[WARNING] PRE_ACT(4100470,7f5f2f443740,python):2021-11-08-12:57:24.272.239 [mindspore/ccsrc/backend/optimizer/gpu/reduce_precision_fusion.cc:83] Run] Reduce precision for [TensorScatterUpdate] input 1
[WARNING] KERNEL(4100470,7f5f2f443740,python):2021-11-08-12:57:24.278.648 [mindspore/ccsrc/backend/kernel_compiler/gpu/gpu_kernel_factory.cc:96] ReducePrecision] Kernel [TensorScatterUpdate] does not support int64, cast input 1 to int32.
[WARNING] PRE_ACT(4100470,7f5f2f443740,python):2021-11-08-12:57:24.278.726 [mindspore/ccsrc/backend/optimizer/gpu/reduce_precision_fusion.cc:83] Run] Reduce precision for [TensorScatterUpdate] input 1


Tensor(shape=[2, 2], dtype=Float32, value=
[[ 5.60000000e+01,  7.20000000e+01],
 [ 1.04000000e+02,  1.20000000e+02]])

计算多个通道的输出的互相关函数

In [4]:
def corr2d_multi_in_out(X, K):
    # 迭代“K”的第0个维度，每次都对输入“X”执行互相关运算。
    # 最后将所有结果都叠加在一起
    return d2l.stack([corr2d_multi_in(X, k) for k in K], 0)

K = d2l.stack((K, K + 1, K + 2), 0)
K.shape

(3, 2, 2, 2)

In [5]:
corr2d_multi_in_out(X, K)

Tensor(shape=[3, 2, 2], dtype=Float32, value=
[[[ 5.60000000e+01,  7.20000000e+01],
  [ 1.04000000e+02,  1.20000000e+02]],
 [[ 7.60000000e+01,  1.00000000e+02],
  [ 1.48000000e+02,  1.72000000e+02]],
 [[ 9.60000000e+01,  1.28000000e+02],
  [ 1.92000000e+02,  2.24000000e+02]]])

1x1卷积

In [6]:
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = d2l.reshape(X, (c_i, h * w))
    K = d2l.reshape(K, (c_o, c_i))
    # 全连接层中的矩阵乘法
    Y = d2l.matmul(K, X)
    return d2l.reshape(Y, (c_o, h, w))

X = d2l.normal((3, 3, 3), 0, 1)
K = d2l.normal((2, 3, 1, 1), 0, 1)

Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
assert float(d2l.reduce_sum(d2l.abs(Y1 - Y2))) < 1e-6

[WARNING] KERNEL(4100470,7f5f2f443740,python):2021-11-08-12:57:24.494.096 [mindspore/ccsrc/backend/kernel_compiler/gpu/gpu_kernel_factory.cc:96] ReducePrecision] Kernel [TensorScatterUpdate] does not support int64, cast input 1 to int32.
[WARNING] PRE_ACT(4100470,7f5f2f443740,python):2021-11-08-12:57:24.494.178 [mindspore/ccsrc/backend/optimizer/gpu/reduce_precision_fusion.cc:83] Run] Reduce precision for [TensorScatterUpdate] input 1
[WARNING] KERNEL(4100470,7f5f2f443740,python):2021-11-08-12:57:24.499.155 [mindspore/ccsrc/backend/kernel_compiler/gpu/gpu_kernel_factory.cc:96] ReducePrecision] Kernel [TensorScatterUpdate] does not support int64, cast input 1 to int32.
[WARNING] PRE_ACT(4100470,7f5f2f443740,python):2021-11-08-12:57:24.499.231 [mindspore/ccsrc/backend/optimizer/gpu/reduce_precision_fusion.cc:83] Run] Reduce precision for [TensorScatterUpdate] input 1
